# LDOS Statistical Analysis

This notebook performs rigorous statistical analysis of experiment results.

## Contents
1. Setup and Data Loading
2. Normality Testing
3. Two-Sample Comparisons (Baseline vs Load)
4. Multi-Group ANOVA
5. Effect Size Analysis
6. Confidence Intervals
7. Power Analysis
8. Export Results

## 1. Setup and Data Loading

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import sys
import warnings

# Statistical imports
from scipy import stats
from scipy.stats import shapiro, levene, ttest_ind, mannwhitneyu, f_oneway, kruskal

try:
    from statsmodels.stats.multicomp import pairwise_tukeyhsd
    from statsmodels.stats.power import TTestIndPower
    HAS_STATSMODELS = True
except ImportError:
    HAS_STATSMODELS = False
    print("Warning: statsmodels not installed. Some tests unavailable.")
    print("Install with: pip install statsmodels")

# Add analysis directory to path
sys.path.insert(0, str(Path('..').resolve() / 'analysis'))

try:
    from stats_utils import (
        confidence_interval, cohens_d, compare_scenarios, 
        anova_with_posthoc, required_sample_size
    )
    HAS_STATS_UTILS = True
except ImportError:
    HAS_STATS_UTILS = False
    print("Note: stats_utils.py not found. Using inline implementations.")

# Configure
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
warnings.filterwarnings('ignore')

# Project paths
WS_ROOT = Path('..').resolve()
RESULTS_DIR = WS_ROOT / 'results'
ANALYSIS_OUTPUT = WS_ROOT / 'analysis' / 'output'

print(f"scipy version: {stats.scipy_version}" if hasattr(stats, 'scipy_version') else f"scipy loaded")
print(f"statsmodels available: {HAS_STATSMODELS}")
print(f"stats_utils available: {HAS_STATS_UTILS}")

In [ ]:
# Inline implementations if stats_utils not available
if not HAS_STATS_UTILS:
    def confidence_interval(data, confidence=0.95):
        """Compute confidence interval for the mean."""
        data = np.array(data)
        n = len(data)
        mean = np.mean(data)
        se = stats.sem(data)
        h = se * stats.t.ppf((1 + confidence) / 2, n - 1)
        return mean, mean - h, mean + h
    
    def cohens_d(group1, group2):
        """Compute Cohen's d effect size."""
        n1, n2 = len(group1), len(group2)
        var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
        pooled_std = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
        return (np.mean(group1) - np.mean(group2)) / pooled_std if pooled_std > 0 else 0

In [ ]:
# Load data (same as exploration notebook)
def load_combined_results(results_dir: Path) -> pd.DataFrame:
    all_rows = []
    
    if not results_dir.exists():
        return pd.DataFrame()
    
    for scenario_dir in results_dir.iterdir():
        if not scenario_dir.is_dir():
            continue
            
        scenario_name = scenario_dir.name
        json_files = list(scenario_dir.glob('*_result.json'))
        
        for json_file in json_files:
            try:
                with open(json_file) as f:
                    data = json.load(f)
                data['scenario'] = scenario_name
                all_rows.append(data)
            except:
                pass
        
        # Fallback to summary.csv
        if not json_files:
            summary_csv = scenario_dir / 'summary.csv'
            if summary_csv.exists():
                try:
                    df_summary = pd.read_csv(summary_csv)
                    df_summary['scenario'] = scenario_name
                    all_rows.extend(df_summary.to_dict('records'))
                except:
                    pass
    
    return pd.DataFrame(all_rows) if all_rows else pd.DataFrame()

df = load_combined_results(RESULTS_DIR)
print(f"Loaded {len(df)} trials from {df['scenario'].nunique() if len(df) > 0 else 0} scenarios")

# Filter to successful trials
if len(df) > 0 and 'status' in df.columns:
    df_success = df[df['status'] == 'success'].copy()
    print(f"Successful trials: {len(df_success)}")
else:
    df_success = df.copy()

## 2. Normality Testing

Before parametric tests, check if data is normally distributed.

In [ ]:
def test_normality(data, name="Data", alpha=0.05):
    """
    Test for normality using Shapiro-Wilk test.
    
    Returns:
        dict with statistic, p_value, and is_normal flag
    """
    data = np.array(data).flatten()
    data = data[~np.isnan(data)]
    
    if len(data) < 3:
        return {'name': name, 'statistic': np.nan, 'p_value': np.nan, 'is_normal': False, 'n': len(data)}
    
    # Shapiro-Wilk (up to 5000 samples)
    if len(data) > 5000:
        data = np.random.choice(data, 5000, replace=False)
    
    stat, p_value = shapiro(data)
    is_normal = p_value > alpha
    
    return {
        'name': name,
        'n': len(data),
        'statistic': stat,
        'p_value': p_value,
        'is_normal': is_normal
    }

if len(df_success) > 0:
    print("=== Normality Tests (Shapiro-Wilk) ===")
    print("H0: Data is normally distributed")
    print("p < 0.05 suggests non-normality\n")
    
    normality_results = []
    
    for col in ['total_latency_ms', 'planning_latency_ms', 'execution_latency_ms']:
        if col not in df_success.columns:
            continue
            
        for scenario in df_success['scenario'].unique():
            data = df_success[df_success['scenario'] == scenario][col].dropna()
            result = test_normality(data, f"{scenario}/{col}")
            normality_results.append(result)
    
    normality_df = pd.DataFrame(normality_results)
    display(normality_df)

In [ ]:
# Q-Q plots for visual normality check
if len(df_success) > 0 and 'total_latency_ms' in df_success.columns:
    col = 'total_latency_ms'
    scenarios = df_success['scenario'].unique()
    n_scenarios = len(scenarios)
    
    fig, axes = plt.subplots(1, n_scenarios, figsize=(5*n_scenarios, 4))
    if n_scenarios == 1:
        axes = [axes]
    
    for ax, scenario in zip(axes, scenarios):
        data = df_success[df_success['scenario'] == scenario][col].dropna()
        stats.probplot(data, dist="norm", plot=ax)
        ax.set_title(f'{scenario}\nQ-Q Plot')
    
    plt.tight_layout()
    plt.show()

## 3. Two-Sample Comparisons (Baseline vs Load)

Compare baseline to each load scenario using appropriate statistical tests.

In [ ]:
def compare_two_groups(group1, group2, name1="Group1", name2="Group2", alpha=0.05):
    """
    Comprehensive comparison of two groups.
    
    Performs:
    - Levene's test for equal variances
    - Two-sample t-test (Welch's if variances unequal)
    - Mann-Whitney U test (non-parametric alternative)
    - Cohen's d effect size
    """
    g1 = np.array(group1).flatten()
    g2 = np.array(group2).flatten()
    g1 = g1[~np.isnan(g1)]
    g2 = g2[~np.isnan(g2)]
    
    results = {
        'comparison': f"{name1} vs {name2}",
        'n1': len(g1),
        'n2': len(g2),
        'mean1': np.mean(g1),
        'mean2': np.mean(g2),
        'std1': np.std(g1, ddof=1),
        'std2': np.std(g2, ddof=1),
    }
    
    # Percent change
    if results['mean1'] > 0:
        results['pct_change'] = (results['mean2'] - results['mean1']) / results['mean1'] * 100
    else:
        results['pct_change'] = np.nan
    
    # Levene's test for equal variances
    levene_stat, levene_p = levene(g1, g2)
    equal_var = levene_p > alpha
    results['levene_p'] = levene_p
    results['equal_variance'] = equal_var
    
    # T-test (Welch's if unequal variance)
    t_stat, t_p = ttest_ind(g1, g2, equal_var=equal_var)
    results['t_statistic'] = t_stat
    results['t_p_value'] = t_p
    results['t_significant'] = t_p < alpha
    
    # Mann-Whitney U (non-parametric)
    u_stat, u_p = mannwhitneyu(g1, g2, alternative='two-sided')
    results['mwu_statistic'] = u_stat
    results['mwu_p_value'] = u_p
    results['mwu_significant'] = u_p < alpha
    
    # Cohen's d
    d = cohens_d(g1, g2)
    results['cohens_d'] = d
    
    # Effect size interpretation
    abs_d = abs(d)
    if abs_d < 0.2:
        results['effect_size'] = 'negligible'
    elif abs_d < 0.5:
        results['effect_size'] = 'small'
    elif abs_d < 0.8:
        results['effect_size'] = 'medium'
    else:
        results['effect_size'] = 'large'
    
    return results

In [ ]:
# Compare baseline to each load scenario
if len(df_success) > 0 and 'total_latency_ms' in df_success.columns:
    col = 'total_latency_ms'
    scenarios = df_success['scenario'].unique()
    
    print("=== Two-Sample Comparisons ===")
    print(f"Metric: {col}\n")
    
    comparison_results = []
    
    if 'baseline' in scenarios:
        baseline_data = df_success[df_success['scenario'] == 'baseline'][col].dropna()
        
        for scenario in scenarios:
            if scenario == 'baseline':
                continue
            
            load_data = df_success[df_success['scenario'] == scenario][col].dropna()
            result = compare_two_groups(baseline_data, load_data, 'baseline', scenario)
            comparison_results.append(result)
        
        if comparison_results:
            comparison_df = pd.DataFrame(comparison_results)
            display(comparison_df[['comparison', 'n1', 'n2', 'mean1', 'mean2', 
                                   'pct_change', 't_p_value', 't_significant',
                                   'cohens_d', 'effect_size']])
    else:
        print("No 'baseline' scenario found. Available:", list(scenarios))

In [ ]:
# Visualize comparison results
if len(df_success) > 0 and 'comparison_results' in dir() and comparison_results:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    comp_df = pd.DataFrame(comparison_results)
    
    # Bar chart of means with error bars
    ax1 = axes[0]
    x = range(len(comp_df))
    width = 0.35
    
    bars1 = ax1.bar([i - width/2 for i in x], comp_df['mean1'], width, 
                    yerr=comp_df['std1'], label='Baseline', capsize=3)
    bars2 = ax1.bar([i + width/2 for i in x], comp_df['mean2'], width,
                    yerr=comp_df['std2'], label='Load', capsize=3)
    
    ax1.set_xticks(x)
    ax1.set_xticklabels([r['comparison'].split(' vs ')[1] for r in comparison_results], rotation=45)
    ax1.set_ylabel('Latency (ms)')
    ax1.set_title('Mean Latency Comparison')
    ax1.legend()
    
    # Add significance stars
    for i, (bar1, bar2, result) in enumerate(zip(bars1, bars2, comparison_results)):
        if result['t_significant']:
            max_height = max(bar1.get_height() + result['std1'], bar2.get_height() + result['std2'])
            ax1.text(i, max_height * 1.05, '*', ha='center', fontsize=16, color='red')
    
    # Effect size bar chart
    ax2 = axes[1]
    colors = ['green' if abs(d) < 0.2 else 'gold' if abs(d) < 0.5 else 'orange' if abs(d) < 0.8 else 'red'
              for d in comp_df['cohens_d']]
    bars = ax2.bar(x, comp_df['cohens_d'], color=colors)
    ax2.axhline(y=0.2, color='green', linestyle='--', alpha=0.5, label='Small (0.2)')
    ax2.axhline(y=0.5, color='orange', linestyle='--', alpha=0.5, label='Medium (0.5)')
    ax2.axhline(y=0.8, color='red', linestyle='--', alpha=0.5, label='Large (0.8)')
    ax2.axhline(y=-0.2, color='green', linestyle='--', alpha=0.5)
    ax2.axhline(y=-0.5, color='orange', linestyle='--', alpha=0.5)
    ax2.axhline(y=-0.8, color='red', linestyle='--', alpha=0.5)
    
    ax2.set_xticks(x)
    ax2.set_xticklabels([r['comparison'].split(' vs ')[1] for r in comparison_results], rotation=45)
    ax2.set_ylabel("Cohen's d")
    ax2.set_title('Effect Sizes (Cohen\'s d)')
    ax2.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

## 4. Multi-Group ANOVA

Compare all scenarios simultaneously using ANOVA with post-hoc tests.

In [ ]:
def run_anova(df: pd.DataFrame, col: str, group_col: str = 'scenario', alpha: float = 0.05):
    """
    Run one-way ANOVA with post-hoc Tukey HSD.
    """
    groups = [df[df[group_col] == g][col].dropna().values for g in df[group_col].unique()]
    group_names = df[group_col].unique().tolist()
    
    # Check if we have enough groups
    if len(groups) < 2:
        print("Need at least 2 groups for ANOVA")
        return None, None
    
    # One-way ANOVA
    f_stat, p_value = f_oneway(*groups)
    
    print("=== One-Way ANOVA ===")
    print(f"F-statistic: {f_stat:.4f}")
    print(f"p-value: {p_value:.6f}")
    print(f"Significant (p < {alpha}): {p_value < alpha}")
    
    # Kruskal-Wallis (non-parametric alternative)
    h_stat, kw_p = kruskal(*groups)
    print(f"\nKruskal-Wallis H-statistic: {h_stat:.4f}")
    print(f"Kruskal-Wallis p-value: {kw_p:.6f}")
    
    # Post-hoc Tukey HSD
    tukey_results = None
    if HAS_STATSMODELS and p_value < alpha:
        print("\n=== Post-Hoc Tukey HSD ===")
        # Prepare data for Tukey
        tukey_data = df[[col, group_col]].dropna()
        tukey_results = pairwise_tukeyhsd(tukey_data[col], tukey_data[group_col], alpha=alpha)
        print(tukey_results)
    
    return {'f_stat': f_stat, 'p_value': p_value, 'h_stat': h_stat, 'kw_p': kw_p}, tukey_results

if len(df_success) > 0 and 'total_latency_ms' in df_success.columns:
    anova_result, tukey = run_anova(df_success, 'total_latency_ms')

In [ ]:
# Visualize Tukey results
if HAS_STATSMODELS and 'tukey' in dir() and tukey is not None:
    fig = tukey.plot_simultaneous(figsize=(10, 6))
    plt.title('Tukey HSD - 95% Confidence Intervals')
    plt.tight_layout()
    plt.show()

## 5. Effect Size Analysis

In [ ]:
def compute_all_effect_sizes(df: pd.DataFrame, col: str, reference: str = 'baseline'):
    """
    Compute effect sizes for all scenarios vs reference.
    """
    scenarios = df['scenario'].unique()
    
    if reference not in scenarios:
        print(f"Reference '{reference}' not found")
        return pd.DataFrame()
    
    ref_data = df[df['scenario'] == reference][col].dropna().values
    
    results = []
    for scenario in scenarios:
        if scenario == reference:
            continue
        
        scenario_data = df[df['scenario'] == scenario][col].dropna().values
        d = cohens_d(ref_data, scenario_data)
        
        # Effect size interpretation
        abs_d = abs(d)
        if abs_d < 0.2:
            interpretation = 'negligible'
        elif abs_d < 0.5:
            interpretation = 'small'
        elif abs_d < 0.8:
            interpretation = 'medium'
        else:
            interpretation = 'large'
        
        results.append({
            'scenario': scenario,
            'cohens_d': d,
            'abs_d': abs_d,
            'interpretation': interpretation,
            'direction': 'increase' if d > 0 else 'decrease'
        })
    
    return pd.DataFrame(results)

if len(df_success) > 0 and 'total_latency_ms' in df_success.columns:
    print("=== Effect Sizes (Cohen's d) ===")
    print("Reference: baseline\n")
    
    effect_df = compute_all_effect_sizes(df_success, 'total_latency_ms')
    if len(effect_df) > 0:
        display(effect_df)

## 6. Confidence Intervals

In [ ]:
def compute_confidence_intervals(df: pd.DataFrame, col: str, confidence: float = 0.95):
    """
    Compute confidence intervals for each scenario.
    """
    results = []
    
    for scenario in df['scenario'].unique():
        data = df[df['scenario'] == scenario][col].dropna().values
        
        if len(data) < 2:
            continue
        
        mean, lower, upper = confidence_interval(data, confidence)
        
        results.append({
            'scenario': scenario,
            'n': len(data),
            'mean': mean,
            'ci_lower': lower,
            'ci_upper': upper,
            'margin': upper - mean,
            'std': np.std(data, ddof=1)
        })
    
    return pd.DataFrame(results)

if len(df_success) > 0 and 'total_latency_ms' in df_success.columns:
    print("=== 95% Confidence Intervals ===")
    
    ci_df = compute_confidence_intervals(df_success, 'total_latency_ms')
    display(ci_df.round(2))

In [ ]:
# Visualize confidence intervals
if len(df_success) > 0 and 'ci_df' in dir() and len(ci_df) > 0:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    x = range(len(ci_df))
    
    ax.errorbar(x, ci_df['mean'], 
                yerr=[ci_df['mean'] - ci_df['ci_lower'], ci_df['ci_upper'] - ci_df['mean']],
                fmt='o', capsize=5, capthick=2, markersize=8)
    
    ax.set_xticks(x)
    ax.set_xticklabels(ci_df['scenario'], rotation=45)
    ax.set_ylabel('Latency (ms)')
    ax.set_title('Mean Latency with 95% Confidence Intervals')
    
    # Add sample sizes
    for i, row in ci_df.iterrows():
        ax.annotate(f'n={row["n"]}', (i, row['ci_upper']), 
                   textcoords='offset points', xytext=(0, 10), ha='center')
    
    plt.tight_layout()
    plt.show()

## 7. Power Analysis

In [ ]:
def power_analysis_sample_size(effect_size: float, power: float = 0.8, alpha: float = 0.05):
    """
    Compute required sample size for desired power.
    """
    if HAS_STATSMODELS:
        analysis = TTestIndPower()
        n = analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha)
        return int(np.ceil(n))
    else:
        # Approximation using z-scores
        z_alpha = stats.norm.ppf(1 - alpha/2)
        z_beta = stats.norm.ppf(power)
        n = 2 * ((z_alpha + z_beta) / effect_size) ** 2
        return int(np.ceil(n))

print("=== Required Sample Sizes for 80% Power ===")
print("(Two-sample t-test, alpha=0.05)\n")

effect_sizes = [0.2, 0.5, 0.8, 1.0, 1.2]
for d in effect_sizes:
    n = power_analysis_sample_size(d)
    label = 'small' if d == 0.2 else 'medium' if d == 0.5 else 'large' if d == 0.8 else ''
    print(f"d = {d:.1f} ({label}): n = {n} per group")

In [ ]:
# Achieved power for current sample sizes
if HAS_STATSMODELS and len(df_success) > 0 and 'comparison_results' in dir() and comparison_results:
    print("\n=== Achieved Power for Current Experiments ===")
    
    analysis = TTestIndPower()
    
    for result in comparison_results:
        d = abs(result['cohens_d'])
        n1, n2 = result['n1'], result['n2']
        
        # Use harmonic mean of sample sizes
        n_eff = 2 * n1 * n2 / (n1 + n2)
        
        if d > 0:
            power = analysis.solve_power(effect_size=d, nobs1=n_eff, alpha=0.05)
            print(f"{result['comparison']}: d={d:.2f}, n_eff={n_eff:.0f}, power={power:.2f}")

## 8. Export Results

In [ ]:
def export_statistical_summary(comparison_df: pd.DataFrame, ci_df: pd.DataFrame, 
                               output_dir: Path):
    """
    Export statistical results to various formats.
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # CSV exports
    comparison_df.to_csv(output_dir / 'statistical_comparisons.csv', index=False)
    ci_df.to_csv(output_dir / 'confidence_intervals.csv', index=False)
    
    print(f"Saved: {output_dir / 'statistical_comparisons.csv'}")
    print(f"Saved: {output_dir / 'confidence_intervals.csv'}")
    
    # LaTeX table for comparison results
    latex_cols = ['comparison', 'pct_change', 't_p_value', 'cohens_d', 'effect_size']
    if all(c in comparison_df.columns for c in latex_cols):
        latex_df = comparison_df[latex_cols].copy()
        latex_df['pct_change'] = latex_df['pct_change'].apply(lambda x: f"{x:+.1f}%")
        latex_df['t_p_value'] = latex_df['t_p_value'].apply(lambda x: f"{x:.4f}" if x >= 0.001 else "<0.001")
        latex_df['cohens_d'] = latex_df['cohens_d'].apply(lambda x: f"{x:.2f}")
        
        latex = latex_df.to_latex(
            index=False,
            caption='Statistical Comparison of Scenarios',
            label='tab:stat_comparison',
            column_format='l' + 'r' * (len(latex_cols) - 1)
        )
        
        with open(output_dir / 'stat_comparison.tex', 'w') as f:
            f.write(latex)
        print(f"Saved: {output_dir / 'stat_comparison.tex'}")

# Uncomment to export
# if 'comparison_results' in dir() and comparison_results and 'ci_df' in dir():
#     export_statistical_summary(pd.DataFrame(comparison_results), ci_df, ANALYSIS_OUTPUT / 'stats')

In [ ]:
# Generate summary report
if len(df_success) > 0 and 'comparison_results' in dir() and comparison_results:
    print("\n" + "="*60)
    print("STATISTICAL ANALYSIS SUMMARY")
    print("="*60)
    
    print(f"\nTotal successful trials: {len(df_success)}")
    print(f"Scenarios analyzed: {df_success['scenario'].nunique()}")
    
    print("\nKey Findings:")
    for result in comparison_results:
        sig = "*" if result['t_significant'] else ""
        print(f"  - {result['comparison']}: {result['pct_change']:+.1f}% change, "
              f"d={result['cohens_d']:.2f} ({result['effect_size']}){sig}")
    
    print("\n* = statistically significant (p < 0.05)")

## Next Steps

1. **`03_path_analysis.ipynb`** - Analyze end-to-end callback chains

### Interpretation Guide

| Effect Size (d) | Interpretation | Typical Impact |
|-----------------|----------------|----------------|
| < 0.2 | Negligible | Hard to notice in practice |
| 0.2 - 0.5 | Small | Noticeable with careful observation |
| 0.5 - 0.8 | Medium | Clearly noticeable |
| > 0.8 | Large | Obvious, substantial impact |

### Statistical Significance vs Practical Significance

- **p < 0.05**: Statistically significant (unlikely due to chance)
- **Cohen's d**: Practical significance (magnitude of difference)
- Both should be considered when interpreting results